In [1]:
import pickle
import xgboost as xgb

In [2]:
with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)

model = xgb.Booster()
model.load_model('model.bin')

In [3]:
dv, model

(DictVectorizer(sparse=False), <xgboost.core.Booster at 0x12ae8474a00>)

In [4]:
code_params = {
    "loc": 33.0, 
    "v(g)": 5.0, 
    "ev(g)": 1.00, "iv(g)": 4.00, 
    "n": 144.00, "v": 824.82, "l":0.04, "d":26.96, "i":30.05, "e":22636.74,
    "b":0.27, "t":1257.60, "lOCode":3.00, "lOComment":0.00, "lOBlank":3.00, 
    "locCodeAndComment":0.00, "uniq_Op":21.00, "uniq_Opnd":23.00, "total_Op":87.00, 
    "total_Opnd":57.00, "branchCount":9.00
}

In [5]:
# transform above dictionary to binary
X = dv.transform(code_params)
X

array([[2.700000e-01, 9.000000e+00, 2.696000e+01, 2.263674e+04,
        1.000000e+00, 3.005000e+01, 4.000000e+00, 4.000000e-02,
        3.000000e+00, 3.000000e+00, 0.000000e+00, 3.300000e+01,
        0.000000e+00, 1.440000e+02, 1.257600e+03, 8.700000e+01,
        5.700000e+01, 2.100000e+01, 2.300000e+01, 8.248200e+02,
        5.000000e+00]])

In [6]:
# transform to Dmatrix for xgboost
X_pred = xgb.DMatrix(X, feature_names=dv.get_feature_names_out().tolist())
X_pred

In [7]:
feature_names=dv.get_feature_names_out()
feature_names.tolist()

['b',
 'branchCount',
 'd',
 'e',
 'ev(g)',
 'i',
 'iv(g)',
 'l',
 'lOBlank',
 'lOCode',
 'lOComment',
 'loc',
 'locCodeAndComment',
 'n',
 't',
 'total_Op',
 'total_Opnd',
 'uniq_Op',
 'uniq_Opnd',
 'v',
 'v(g)']

In [8]:
y_pred = model.predict(X_pred)
y_pred

array([0.24367875], dtype=float32)

In [9]:
result = {'defect': y_pred}

In [10]:
print(result)

{'defect': array([0.24367875], dtype=float32)}


In [11]:
type(result)

dict

In [18]:
result = {'defect': y_pred}
print(result)

{'defect': array([0.24367875], dtype=float32)}


### Test service

In [13]:
import requests

In [24]:
url = "http://localhost:9696/predict"

In [27]:
requests.post(url, json=code_params)

ConnectionError: HTTPConnectionPool(host='localhost', port=9696): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000012AE84B7FD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [26]:
response = requests.post(url, json=code_params)
response.json()

{'defect': 0.2436787486076355}